In [1]:
import pandas as pd
import yfinance as yf
import requests
import time
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)
db = client.db

In [3]:
df = pd.DataFrame(columns = ['created_utc', 'body', 'score'])

In [4]:
def getData(datatype, **kwargs):
    url = f'https://api.pushshift.io/reddit/search/{datatype}/'
    payload = kwargs
    req = requests.get(url, params = payload)
    return req.json()['data']

In [5]:
subreddit = 'wallstreetbets'
lastoctober = 1569888000
november = 1572566400
december = 1575158400
january = 1577836800
february = 1580515200
march = 1583020800
april = 1585699200
may = 1588291200
june = 1590969600
july = 1593561600
august = 1596240000
september = 1598918400
thisoctober = 1601510400
size = 500

In [6]:
df = pd.DataFrame()

In [7]:
after = lastoctober
before = thisoctober
while before > after:
    testdict = getData(datatype = 'submission', subreddit = subreddit, after = after, q = 'Daily Discussion Thread', before = before, size = size)
    if not testdict:
        break
    tempdf = pd.DataFrame.from_records(testdict)
    df = df.append(tempdf, ignore_index = True)
    after = df['created_utc'][len(df)-1]
    print(after)
    time.sleep(1)

1575630325
1581368695
1586370524
1591733184
1596829613
1601496415


In [10]:
df['title']

0            Daily Discussion Thread - October 01, 2019
1              What Are Your Moves Tomorrow, October 02
2            Daily Discussion Thread - October 02, 2019
3              What Are Your Moves Tomorrow, October 03
4            Daily Discussion Thread - October 03, 2019
                             ...                       
586    What Are Your Moves Tomorrow, September 29, 2020
587      Daily Discussion Thread for September 29, 2020
588    What Are Your Moves Tomorrow, September 30, 2020
589      Daily Discussion Thread for September 30, 2020
590      What Are Your Moves Tomorrow, October 01, 2020
Name: title, Length: 591, dtype: object

In [6]:
df = pd.DataFrame()

In [7]:
after = 1599362923
before = thisoctober
while before > after:
    testdict = getData(datatype = 'comment', subreddit = subreddit, after = after, before = before, size = size)
    if not testdict:
        break
    tempdf = pd.DataFrame.from_records(testdict)
    df = df.append(tempdf, ignore_index = True)
    after = df['created_utc'][len(df)-1]
    print(after)
    time.sleep(1)

6
1599132801
1599132980
1599133175
1599133335
1599133519
1599133701
1599133887
1599134054
1599134235
1599134386
1599134551
1599134716
1599134880
1599135012
1599135151
1599135314
1599135458
1599135593
1599135713
1599135807
1599135929
1599136062
1599136202
1599136305
1599136410
1599136543
1599136681
1599136781
1599136876
1599136986
1599137089
1599137227
1599137341
1599137449
1599137556
1599137672
1599137773
1599137877
1599137981
1599138086
1599138201
1599138312
1599138403
1599138503
1599138574
1599138647
1599138723
1599138793
1599138871
1599138941
1599139010
1599139083
1599139165
1599139224
1599139296
1599139371
1599139432
1599139505
1599139574
1599139637
1599139696
1599139745
1599139790
1599139857
1599139940
1599140025
1599140091
1599140162
1599140231
1599140305
1599140383
1599140438
1599140509
1599140568
1599140646
1599140720
1599140792
1599140857
1599140910
1599140969
1599141041
1599141112
1599141176
1599141236
1599141292
1599141357
1599141420
1599141484
1599141554
1599141619
15991416

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
df['created_utc']

0         1599094503
1         1599094503
2         1599094504
3         1599094505
4         1599094507
             ...    
201795    1599362907
201796    1599362913
201797    1599362922
201798    1599362923
201799    1599362923
Name: created_utc, Length: 201800, dtype: int64

In [11]:
uploaddf = df[['id', 'created_utc', 'score', 'body']]

In [12]:
uploaddf = uploaddf.loc[uploaddf['score'] > 1]

In [13]:
uploaddf = uploaddf.loc[~uploaddf['body'].isin(['[deleted]', '[removed]'])]

In [14]:
uploaddf = uploaddf.reset_index(drop = True)

In [15]:
uploaddf

,id,created_utc,score,body
0,g3sjoe8,1599094503,4,Why would you sell now? 🧐
1,g3sjoij,1599094505,2,Tesla 475 call. Down 80% and it expires this f...
2,g3sjphh,1599094518,4,The real DD I should have followed on RKT is i...
3,g3sjpk0,1599094520,4,I will never financially recover from this
4,g3sjprh,1599094523,4,Limit down on futures is .5% right?
...,...,...,...,...
84861,g46zc9u,1599358378,2,\n\n# Facebook rolls out TikTok rival Instagr...
84862,g46zk1z,1599358513,2,"I lost a lot of money on Thursday on Friday, o..."
84863,g46zmxx,1599358564,2,Sooooo what Tesla gonna do this coming week 🤔
84864,g470l7m,1599359171,2,nah it's the weekend


In [16]:
db.comments.insert_many(uploaddf.to_dict('records'))